## 动态规划 Dymanic Programming

### 切钢筋问题

In [104]:
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30]

from collections import defaultdict
price = defaultdict(int)

for i, p in enumerate(original_price):
    price[i+1]=p


from functools import wraps
called_time=defaultdict(int)
def get_call_times(f):
    @wraps(f)
    def wrap(n):
        result = f(n)
        called_time[(f.__name__,n)]+=1
        return result
      
    return wrap

def memo(f):
    memo.already_computed = {}
    @wraps(f)
    def _wrap(arg):
        result = None
        if arg in memo.already_computed:
            result = memo.already_computed[arg]
        else:
            result = f(arg)
            memo.already_computed[arg]=result
        return result
    return _wrap

@memo
@get_call_times
def r(n):
    return max(
        [price[n]] +[r(i) + r(n-i) for i in range(1,n)]
    )

print(r(9))

from collections import Counter

Counter(called_time).most_common()

25


[(('r', 1), 1),
 (('r', 2), 1),
 (('r', 3), 1),
 (('r', 4), 1),
 (('r', 5), 1),
 (('r', 6), 1),
 (('r', 7), 1),
 (('r', 8), 1),
 (('r', 9), 1)]

In [105]:
solution={}
@memo
def split(n):
    max_price, max_split = max(
        [(price[n],0)] + [(split(i)+split(n-i),i) for i in range(1,n)], key=lambda x:x[0]
    )
    solution[n] = (n - max_split, max_split)
    return max_price
    
def parse_solution(n):
    left_split, right_split = solution[n]
    if right_split == 0:
        return [left_split]
    #list拼接
    return parse_solution(left_split)+parse_solution(right_split)

print(split(19))
print(solution)
parse_solution(19)

55
{1: (1, 0), 2: (2, 0), 3: (3, 0), 4: (2, 2), 5: (3, 2), 6: (6, 0), 7: (6, 1), 8: (6, 2), 9: (6, 3), 10: (10, 0), 11: (10, 1), 12: (10, 2), 13: (10, 3), 14: (12, 2), 15: (13, 2), 16: (10, 6), 17: (16, 1), 18: (16, 2), 19: (16, 3)}


[10, 6, 3]

### 编辑距离问题

In [106]:
solution = {}

In [107]:
from functools import lru_cache 
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    #特殊情况直接得出结果
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    #末尾对比
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    #candidates记录当前所以可能的子操作
    #分解子问题，对于长度不同情况，要么左边删除末尾得到右边，要么左边增加末尾得到右边
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  # string 1 add tail of string2
    ]
    
    #末尾相同，则左右字符串进一步对比去除末尾的其余字符串
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    #末尾不同，则左右字符串通过修改末尾从左边得到右边，再进一步对比去除末尾的其余字符串
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    #分解子问题，继续记录当前可能的子操作
    candidates.append(both_forward)
    
    #在当前所以可能子操作中求最短的编辑路径
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    #将当前状态的最佳操作记录起来
    solution[(string1, string2)] = operation 
    
    return min_distance

In [120]:
def parse_solution(a,b):
    action = solution[(a,b)]
    if a==b and action == '':
        return []
    if 'ADD' in action:
        next_a=a
        next_b=b[:-1]
    if 'DEL' in action:
        next_a=a[:-1]
        next_b=b
    if 'SUB' in action:
        next_a=a[:-1]
        next_b=b[:-1]
    if a!=b and action =='':
        next_a=a[:-1]
        next_b=b[:-1]
    #list拼接
    return parse_solution(next_a, next_b)+[action]


In [121]:
print(edit_distance('ABBEEF', 'ABCCEF'))
print(solution)
parse_solution('ABBEEF', 'ABCCEF')

2
{('A', 'A'): '', ('A', 'AB'): 'ADD B', ('A', 'ABC'): 'ADD C', ('A', 'ABCC'): 'ADD C', ('A', 'ABCCE'): 'ADD E', ('A', 'ABCCEF'): 'ADD F', ('AB', 'A'): 'DEL B', ('AB', 'AB'): '', ('AB', 'ABC'): 'ADD C', ('AB', 'ABCC'): 'ADD C', ('AB', 'ABCCE'): 'ADD E', ('AB', 'ABCCEF'): 'ADD F', ('ABB', 'A'): 'DEL B', ('ABB', 'AB'): 'DEL B', ('ABB', 'ABC'): 'SUB B => C', ('ABB', 'ABCC'): 'ADD C', ('ABB', 'ABCCE'): 'ADD E', ('ABB', 'ABCCEF'): 'ADD F', ('ABBE', 'A'): 'DEL E', ('ABBE', 'AB'): 'DEL E', ('ABBE', 'ABC'): 'DEL E', ('ABBE', 'ABCC'): 'SUB E => C', ('ABBE', 'ABCCE'): '', ('ABBE', 'ABCCEF'): 'ADD F', ('ABBEE', 'A'): 'DEL E', ('ABBEE', 'AB'): 'DEL E', ('ABBEE', 'ABC'): 'DEL E', ('ABBEE', 'ABCC'): 'DEL E', ('ABBEE', 'ABCCE'): '', ('ABBEE', 'ABCCEF'): 'ADD F', ('ABBEEF', 'A'): 'DEL F', ('ABBEEF', 'AB'): 'DEL F', ('ABBEEF', 'ABC'): 'DEL F', ('ABBEEF', 'ABCC'): 'DEL F', ('ABBEEF', 'ABCCE'): 'DEL F', ('ABBEEF', 'ABCCEF'): ''}


['SUB B => C', 'SUB E => C', '', '']

## 动态规划QA

1. 为什么我们需要动态规划？动态规划和搜索方法的区别是什么？

原先的搜索方法遇到重复的子问题时不会将解保存起来，导致计算效率低。为了提高效率，我们用动态规划，通过表存储重复子问题的解。

2. 同样的问题，为什么我们需要动态规划？为什么不通过训练一个机器学习模型来根据输入得到“正确”的答案？

因为机器学习通常需要大量数据，而动态规划解决的问题往往只有有限的数据，需要从有限数据找出规律，因此需要我们将问题抽象出重复的子问题进行求解。

3. 举出三个可以用动态规划解决的问题？

1. 资源分配问题
2. 字符串匹配问题（如编辑距离）
3. 最短路径问题（旅行商问题）

4. 举出三个可以用编辑距离解决的问题？

1. 单词相似度计算
2. 单词纠错
3. 输入自动补全

5. 总结动态规划的三个主要特点，并且简要描述每个特点。

1. 原问题**可以**分解为重叠的子问题。重叠子问题的本质就是待研究问题对象的不同状态，彼此之间存在转移关系，如何分解即我们要寻找的状态转移方程。
2. 重叠的子问题的解**可以**记录到一个表中。表的key存储方程参数，value存储方程结果。
3. 原问题的最终解**可以**通过不断查表得到。不断查表求解就是把原问题拆解后的所有子问题的解合并起来。

6. 动态规划的缺点是什么？

缺点是存在维数灾难问题。因为每次都要对所有子问题的状态进行遍历，当状态非常大的适合计算量就非常大。